In [13]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

In [14]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
scratchDirectory='/mnt/lustre/koa/scratch/air673/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/CodeFiles/Project_Algorithms/Domain_Profiles/'

In [15]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [16]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [17]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [18]:
################################################################################################################################################################

In [19]:
#FROM MOISTURE FLUX CONVERGENCE: ITS HISTORY AND APPLICATION IN CONVECTIVE INITIATION FORECASTING
# Peter C. Banacos
# NOAA/NWS/NCEP/Storm Prediction Center
# Norman, Oklahoma 73069
# David M. Schultz
# Cooperative Institute for Mesoscale Meteorological Studies, Univ. of Oklahoma, and
# NOAA/National Severe Storms Laboratory, Norman, Oklahoma 73069

In [20]:
#MAKING ARRAY TO STORE VARIABLE
# DIRECT CALCULATION
# WITHOUT ADVECTION + CONVERGENCE VECTOR IDENTIY

In [ ]:
if res=='1km':
    outputDirectory=os.path.join(mainDirectory,'Code','OUTPUT','Variable_Calculation','CalculateMoreVariables')
    os.makedirs(outputDirectory, exist_ok=True)
if res=='250m':
    outputDirectory=os.path.join(scratchDirectory,'OUTPUT','Variable_Calculation','CalculateMoreVariables')
    os.makedirs(outputDirectory, exist_ok=True)
    
def initiate_array():
    # Define array dimensions (adjust based on your netCDF)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file=os.path.join(outputDirectory, '2D_Moisture_Convergence'+f'_{res}_{t_res}'+'.h5')
    with h5py.File(out_file, 'a') as f:
        # Check if the dataset 'theta_e' already exists
        if 'HMC' not in f:
            # Create a dataset with the full size for all time steps (initially empty)
            f.create_dataset('HMC', 
                             (t_size, z_size, y_size, x_size),  # Full size for all timesteps
                             maxshape=(None, z_size, y_size, x_size),  # Unlimited timesteps (can grow along time dimension)
                             dtype='float64', 
                             chunks=(1, z_size, y_size, x_size))  # Chunks for time axis to allow resizing

            
def add_timestep_at_index(timestep_data, index):
    out_file=os.path.join(outputDirectory, '2D_Moisture_Convergence'+f'_{res}_{t_res}'+'.h5')
    with h5py.File(out_file, 'a') as f:
        # Access the existing dataset 'theta_e'
        dataset = f['HMC']
        
        # Assign the new timestep data at the specified index
        dataset[index] = timestep_data

In [ ]:
####################################
#RUNNING

In [ ]:
if res=='1km':
    dx=1000
    dy=1000
elif res=='250m':
    dx=250
    dy=250

In [ ]:
initiate_array()

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')
    
    u_data=data['uinterp'].isel(time=t).data
    v_data=data['vinterp'].isel(time=t).data
    qv_data=data['qv'].isel(time=t).data
    rho_data=data['rho'].isel(time=t).data
        
    print('calculating convergence and taking mean')
    one=Ddx_3D(rho_data*qv_data*u_data,dx)
    two=Ddy_3D(rho_data*qv_data*v_data,dy)
    HMC=-(one+two)
    add_timestep_at_index(HMC, t)


In [ ]:
######################################################

In [ ]:
# #READING BACK IN
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# file_path = dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence' + f'_{res}_{t_res}' + '.h5'
# with h5py.File(file_path, 'r') as f:
#     Conv2 = f['HMC'][:]

In [ ]:
######################################################
#TESTING

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.ticker import ScalarFormatter

# t=100;z=3

# # Assuming Conv and data are already defined and properly loaded

# # Create a 2x3 subplot grid using GridSpec
# fig = plt.figure(figsize=(15, 10))
# gs = fig.add_gridspec(2, 3)

# # Plot 1: Contourf for Conv[t, z]
# ax1 = fig.add_subplot(gs[0, 0])
# MFC_2D2 = Conv2[t, z] 
# c1 = ax1.contourf(MFC_2D2* 1000,levels=20)
# ax1.set_xlabel('x grid')
# ax1.set_ylabel('y grid')
# ax1.set_title('MFC(t, z, :, :)')

# # Plot 2: Contourf for MFC_tz (mean over axis (2, 3))
# ax2 = fig.add_subplot(gs[0, 1])
# MFC_tz2 = np.mean(Conv2, axis=(2, 3))
# c2 = ax2.contourf(MFC_tz2.T * 1000,levels=20)
# ax2.set_xlabel('t grid')
# ax2.set_ylabel('z grid')
# ax2.set_title('MFC(t, z)')

# # Plot 3: Contourf for MFC_tz (mean over axis (0, 2))
# ax3 = fig.add_subplot(gs[0, 2])
# MFC_zx2 = np.mean(Conv2, axis=(0, 2))
# c3 = ax3.contourf(MFC_zx2 * 1000,levels=20)
# ax3.set_xlabel('x grid')
# ax3.set_ylabel('z grid')
# ax3.set_title('MFC(z,x)')

# # Plot 4: Contourf for MFC_tz (mean over axis (0, 3))
# ax4 = fig.add_subplot(gs[1, 0])
# MFC_zy2 = np.mean(Conv2, axis=(0, 3))
# c4 = ax4.contourf(MFC_zy2 * 1000,levels=20)
# ax4.set_xlabel('y grid')
# ax4.set_ylabel('z grid')
# ax4.set_title('MFC(z,y)')

# # Plot 5: Plot of MFC_z (mean over axis (0, 2, 3))
# ax5 = fig.add_subplot(gs[1, 1])
# MFC_z2 = np.mean(Conv2, axis=(0, 2, 3))
# ax5.plot(MFC_z2 * 1000, data['zh'])
# ax5.set_xlabel('2D MFC (g/kg/s)')
# ax5.set_ylabel('z (km)')
# ax5.set_title('MFC(z)')

# # Plot 6: Time plot of MFC_z (mean over axis (1, 2, 3))
# ax6 = fig.add_subplot(gs[1, 2])
# MFC_t2 = np.mean(Conv2, axis=(1, 2, 3))
# ts = np.arange(len(data['time']))
# ax6.plot(ts, MFC_t2 * 1000)
# ax6.set_xlabel('time (5 mins)')
# ax6.set_ylabel('2D MFC (g/kg/s)')
# ax6.set_title('MFC(t)')

# # Add colorbars with scientific notation
# cb1 = fig.colorbar(c1, ax=ax1, label='g/kg/s')
# cb2 = fig.colorbar(c2, ax=ax2, label='g/kg/s')
# cb3 = fig.colorbar(c3, ax=ax3, label='g/kg/s')
# cb4 = fig.colorbar(c4, ax=ax4, label='g/kg/s')

# # Apply scientific notation to the colorbar
# for cb in [cb1, cb2, cb3, cb4]:
#     cb.formatter = ScalarFormatter()
#     cb.formatter.set_powerlimits((-3, 4))  # Set the range for scientific notation
#     cb.update_ticks()

# # Adjust layout for better spacing
# plt.tight_layout()

In [ ]:
# #PROFILES AT SINGLE TIME

# t=300
# t=100

# #READING BACK IN
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# file_path = dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence' + f'_{res}_{t_res}' + '.h5'
# with h5py.File(file_path, 'r') as f:
#     HMC = f['HMC'][t]

# HMC_z = np.mean(HMC,axis=(1,2))
# mask1 = (data['winterp'][t] >= 0.5) & ((data['qc'][t] + data['qi'][t]) >= 1e-6)
# print(np.any(mask1))
# # masked_profile1 = np.ma.masked_where(~mask1, HMC)
# masked_profile1 = np.where(~mask1, np.nan, HMC)
# HMC_cloudy= np.nanmean(masked_profile1, axis=(1, 2))
# print('done')

# #########################


# # New Subplots for Contour Plots
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# #First plot
# axes[0].plot(HMC_z*1000,data['zh'])
# axes[0].axvline(0,color='black')
# axes[0].set_title('HMC(z) Horizontal Average for Eulerian Data Everywhere',fontsize=8)




# #Second plot
# axes[1].plot(HMC_cloudy*1000,data['zh'])
# axes[1].axvline(0,color='black')
# axes[1].set_title('HMC(z) Horizontal Average for Eulerian Data (w ≥ 0.5) & (qc+qi ≥ 1e-6)',fontsize=8)


# apply_scientific_notation([axes[0],axes[1]])
# plt.suptitle(f'Horizontal Average of 2D HMC For Entire Domain vs Within Cloudy Updrafts (AT TIME {t})')
# # plt.tight_layout()

In [ ]:
# #PROFILES AT SINGLE TIME

# t=300
# t=100

# #READING BACK IN
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# file_path = dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence' + f'_{res}_{t_res}' + '.h5'
# with h5py.File(file_path, 'r') as f:
#     HMC = f['HMC'][t]

# HMC_z = np.mean(HMC,axis=(1,2))
# mask1 = (data['winterp'][t] >= 0.1) & ((data['qc'][t] + data['qi'][t]) < 1e-6)
# print(np.any(mask1))
# # masked_profile1 = np.ma.masked_where(~mask1, HMC)
# masked_profile1 = np.where(~mask1, np.nan, HMC)
# HMC_cloudy= np.nanmean(masked_profile1, axis=(1, 2))
# print('done')

# #########################


# # New Subplots for Contour Plots
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# #First plot
# axes[0].plot(HMC_z*1000,data['zh'])
# axes[0].axvline(0,color='black')
# axes[0].set_title('HMC(z) Horizontal Average for Eulerian Data Everywhere',fontsize=8)




# #Second plot
# axes[1].plot(HMC_cloudy*1000,data['zh'])
# axes[1].axvline(0,color='black')
# axes[1].set_title('HMC(z) Horizontal Average for Eulerian Data (w ≥ 0.1) & (qc+qi < 1e-6)',fontsize=8)


# apply_scientific_notation([axes[0],axes[1]])
# plt.suptitle(f'Horizontal Average of 2D HMC For Entire Domain vs Within General Updrafts (AT TIME {t})')
# # plt.tight_layout()

In [ ]:
# #PROFILES AT AT ALL TIMES
# #READING BACK IN
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# file_path = dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence' + f'_{res}_{t_res}' + '.h5'
# with h5py.File(file_path, 'r') as f:
#     HMC = f['HMC'][:]

# HMC_z = np.mean(HMC,axis=(0,2,3))


# mask1 = (data['winterp'] >= 0.5) & ((data['qc'] + data['qi']) >= 1e-6)
# # masked_profile1 = np.ma.masked_where(~mask1, HMC)
# masked_profile1 = np.where(~mask1, np.nan, HMC)
# HMC_cloudy= np.nanmean(masked_profile1, axis=(0, 2, 3))
# print('done')

# ################################################


# # New Subplots for Contour Plots
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# #First plot
# axes[0].plot(HMC_z*1000,data['zh'])
# axes[0].axvline(0,color='black')
# axes[0].set_title('HMC(z) Horizontal Average for Eulerian Data Everywhere',fontsize=8)


# #Second plot
# axes[1].plot(HMC_cloudy*1000,data['zh'])
# axes[1].axvline(0,color='black')
# axes[1].set_title('HMC(z) Horizontal Average for Eulerian Data (w ≥ 0.5) & (qc+qi ≥ 1e-6)',fontsize=8)


# apply_scientific_notation([axes[0],axes[1]])
# plt.suptitle('Horizontal Average of 2D HMC For Entire Domain vs Within Cloudy Updrafts (ALL TIMES)')
# # plt.tight_layout()

In [ ]:
# def HMC_CloudyProfile(ax, t, data):
#     #READING BACK IN
#     dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
#     file_path = dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence' + f'_{res}_{t_res}' + '.h5'
#     with h5py.File(file_path, 'r') as f:
#         HMC = f['HMC'][t]
    
#     HMC_z = np.mean(HMC,axis=(1,2))
#     mask1 = (data['winterp'][t] >= 0.5) & ((data['qc'][t] + data['qi'][t]) >= 1e-6)
#     # masked_profile1 = np.ma.masked_where(~mask1, HMC)
#     masked_profile1 = np.where(~mask1, np.nan, HMC)
#     import warnings
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore", category=RuntimeWarning)
#         HMC_cloudy = np.nanmean(masked_profile1, axis=(1, 2))
    
#     ax.plot(HMC_cloudy * 1000, data['zh'],color='blue')
#     ax.axvline(0,color='black')
#     ax.axhline(1,color='purple',linestyle='dashed')
#     ax.set_title(f"Time = {t}")
#     apply_scientific_notation([ax])

#     ax.set_ylim(bottom=0)
#     return ax

# fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(15, 10))
# axes = axes.flatten()
# times = np.arange(0, 130, 10)

# for idx, t in enumerate(times):
#     HMC_CloudyProfile(axes[idx], t, data)

# plt.tight_layout()

In [ ]:
########################################################################
#2D MOISTURE FLUX CONVERGENCE CALCULATION
#MOISTURE ADVECTION + MOISTURE * DIVERGENCE

In [ ]:
# if res=='1km':
#     dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# if res=='250m':
#     dir2='/mnt/lustre/koa/scratch/air673/'
# def initiate_array():
#     # Define array dimensions (adjust based on your netCDF)
#     t_size = len(netCDF['time'])  # Number of timesteps
#     z_size = len(netCDF['zh'])    # Number of vertical levels
#     y_size = len(netCDF['yh'])    # Number of y-axis points
#     x_size = len(netCDF['xh'])    # Number of x-axis points
    
#     with h5py.File(dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence'+f'_{res}_{t_res}_2'+'.h5', 'a') as f:
#         # Check if the dataset 'theta_e' already exists
#         if 'HMC' not in f:
#             # Create a dataset with the full size for all time steps (initially empty)
#             f.create_dataset('HMC', 
#                              (t_size, z_size, y_size, x_size),  # Full size for all timesteps
#                              maxshape=(None, z_size, y_size, x_size),  # Unlimited timesteps (can grow along time dimension)
#                              dtype='float64', 
#                              chunks=(1, z_size, y_size, x_size))  # Chunks for time axis to allow resizing

            
# def add_timestep_at_index(timestep_data, index):
#     with h5py.File(dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence'+f'_{res}_{t_res}_2'+'.h5', 'a') as f:
#         # Access the existing dataset 'theta_e'
#         dataset = f['HMC']
        
#         # Assign the new timestep data at the specified index
#         dataset[index] = timestep_data

In [ ]:
# #MAKING ARRAY TO STORE MFC
# # WITHOUT ADVECTION + CONVERGENCE IDENTIY
# initiate_array()

# #CALCULATING AND APPENDING TO DATA EACH TIMESTEP
# for t in range(len(netCDF['time'])):
#     if np.mod(t,1)==0: print(f'Current time {t}')
#     if Np_str=='125e3':
#         u_data=data['u'].isel(time=t).interp(xf=data['xh']).data
#         v_data=data['v'].isel(time=t).interp(yf=data['yh']).data
#         qv_data=data['qv'].isel(time=t).data
#         rho_data=data['rho'].isel(time=t).data
#     elif Np_str=='1e6':
#         u_data=data['uinterp'].isel(time=t).data
#         v_data=data['vinterp'].isel(time=t).data
#         qv_data=data['qv'].isel(time=t).data
#         rho_data=data['rho'].isel(time=t).data
        
#     print('calculating convergence and taking mean')
#     one=u_data*Ddx_3D(rho_data*qv_data,1000)
#     two=v_data*Ddy_3D(rho_data*qv_data,1000)
    
#     four=rho_data*qv_data*Ddx_3D(u_data,1000)
#     five=rho_data*qv_data*Ddy_3D(v_data,1000)
    
#     HMC=-(one+two)-(four+five)
#     add_timestep_at_index(HMC, t)


In [ ]:
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# file_path = dir2 + 'Variable_Calculation/' + '2D_Moisture_Convergence' + f'_{res}_{t_res}_2' + '.h5'
# with h5py.File(file_path, 'r') as f:
#     Conv = f['HMC'][:]

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.ticker import ScalarFormatter

# t=100;z=3

# # Assuming Conv and data are already defined and properly loaded

# # Create a 2x3 subplot grid using GridSpec
# fig = plt.figure(figsize=(15, 10))
# gs = fig.add_gridspec(2, 3)

# # Plot 1: Contourf for Conv[t, z]
# ax1 = fig.add_subplot(gs[0, 0])
# MFC_2D = Conv[t, z] 
# c1 = ax1.contourf(MFC_2D* 1000)
# ax1.set_xlabel('x grid')
# ax1.set_ylabel('y grid')
# ax1.set_title('MFC(t, z, :, :)')

# # Plot 2: Contourf for MFC_tz (mean over axis (2, 3))
# ax2 = fig.add_subplot(gs[0, 1])
# MFC_tz = np.mean(Conv, axis=(2, 3))
# c2 = ax2.contourf(MFC_tz.T * 1000)
# ax2.set_xlabel('t grid')
# ax2.set_ylabel('z grid')
# ax2.set_title('MFC(t, z)')

# # Plot 3: Contourf for MFC_tz (mean over axis (0, 2))
# ax3 = fig.add_subplot(gs[0, 2])
# MFC_zx = np.mean(Conv, axis=(0, 2))
# c3 = ax3.contourf(MFC_zx * 1000)
# ax3.set_xlabel('x grid')
# ax3.set_ylabel('z grid')
# ax3.set_title('MFC(z,x)')

# # Plot 4: Contourf for MFC_tz (mean over axis (0, 3))
# ax4 = fig.add_subplot(gs[1, 0])
# MFC_zy = np.mean(Conv, axis=(0, 3))
# c4 = ax4.contourf(MFC_zy * 1000)
# ax4.set_xlabel('y grid')
# ax4.set_ylabel('z grid')
# ax4.set_title('MFC(z,y)')

# # Plot 5: Plot of MFC_z (mean over axis (0, 2, 3))
# ax5 = fig.add_subplot(gs[1, 1])
# MFC_z = np.mean(Conv, axis=(0, 2, 3))
# ax5.plot(MFC_z * 1000, data['zh'])
# ax5.set_xlabel('2D MFC (g/kg/s)')
# ax5.set_ylabel('z (km)')
# ax5.set_title('MFC(z)')

# # Plot 6: Time plot of MFC_z (mean over axis (1, 2, 3))
# ax6 = fig.add_subplot(gs[1, 2])
# MFC_t = np.mean(Conv, axis=(1, 2, 3))
# ts = np.arange(len(data['time']))
# ax6.plot(ts, MFC_t * 1000)
# ax6.set_xlabel('time (5 mins)')
# ax6.set_ylabel('2D MFC (g/kg/s)')
# ax6.set_title('MFC(t)')

# # Add colorbars with scientific notation
# cb1 = fig.colorbar(c1, ax=ax1, label='g/kg/s')
# cb2 = fig.colorbar(c2, ax=ax2, label='g/kg/s')
# cb3 = fig.colorbar(c3, ax=ax3, label='g/kg/s')
# cb4 = fig.colorbar(c4, ax=ax4, label='g/kg/s')

# # Apply scientific notation to the colorbar
# for cb in [cb1, cb2, cb3, cb4]:
#     cb.formatter = ScalarFormatter()
#     cb.formatter.set_powerlimits((-3, 4))  # Set the range for scientific notation
#     cb.update_ticks()

# # Adjust layout for better spacing
# plt.tight_layout()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.ticker import ScalarFormatter

# t=100;z=3

# # Assuming Conv and data are already defined and properly loaded

# # Create a 2x3 subplot grid using GridSpec
# fig = plt.figure(figsize=(15, 10))
# gs = fig.add_gridspec(2, 3)

# # Plot 1: Contourf for Conv[t, z]
# ax1 = fig.add_subplot(gs[0, 0])
# MFC_2D3 = MFC_2D2-MFC_2D
# c1 = ax1.contourf(MFC_2D3* 1000)
# ax1.set_xlabel('x grid')
# ax1.set_ylabel('y grid')
# ax1.set_title('MFC(t, z, :, :)')

# # Plot 2: Contourf for MFC_tz (mean over axis (2, 3))
# ax2 = fig.add_subplot(gs[0, 1])
# MFC_tz3 = MFC_tz2-MFC_tz
# c2 = ax2.contourf(MFC_tz3.T * 1000)
# ax2.set_xlabel('t grid')
# ax2.set_ylabel('z grid')
# ax2.set_title('MFC(t, z)')

# # Plot 3: Contourf for MFC_tz (mean over axis (0, 2))
# ax3 = fig.add_subplot(gs[0, 2])
# MFC_zx3=MFC_zx2-MFC_zx
# c3 = ax3.contourf(MFC_zx3 * 1000)
# ax3.set_xlabel('x grid')
# ax3.set_ylabel('z grid')
# ax3.set_title('MFC(z,x)')

# # Plot 4: Contourf for MFC_tz (mean over axis (0, 3))
# ax4 = fig.add_subplot(gs[1, 0])
# MFC_zy3 = MFC_zy2-MFC_zy
# c4 = ax4.contourf(MFC_zy3 * 1000)
# ax4.set_xlabel('y grid')
# ax4.set_ylabel('z grid')
# ax4.set_title('MFC(z,y)')

# # Plot 5: Plot of MFC_z (mean over axis (0, 2, 3))
# ax5 = fig.add_subplot(gs[1, 1])
# MFC_z3 =MFC_z2-MFC_z
# ax5.plot(MFC_z3 * 1000, data['zh'])
# ax5.set_xlabel('2D MFC (g/kg/s)')
# ax5.set_ylabel('z (km)')
# ax5.set_title('MFC(z)')

# # Plot 6: Time plot of MFC_z (mean over axis (1, 2, 3))
# ax6 = fig.add_subplot(gs[1, 2])
# MFC_t3 = MFC_t2 - MFC_t
# ts = np.arange(len(data['time']))
# ax6.plot(ts, MFC_t3 * 1000)
# ax6.set_xlabel('time (5 mins)')
# ax6.set_ylabel('2D MFC (g/kg/s)')
# ax6.set_title('MFC(t)')

# # Add colorbars with scientific notation
# cb1 = fig.colorbar(c1, ax=ax1, label='g/kg/s')
# cb2 = fig.colorbar(c2, ax=ax2, label='g/kg/s')
# cb3 = fig.colorbar(c3, ax=ax3, label='g/kg/s')
# cb4 = fig.colorbar(c4, ax=ax4, label='g/kg/s')

# # Apply scientific notation to the colorbar
# for cb in [cb1, cb2, cb3, cb4]:
#     cb.formatter = ScalarFormatter()
#     cb.formatter.set_powerlimits((-3, 4))  # Set the range for scientific notation
#     cb.update_ticks()

# # Adjust layout for better spacing
# plt.tight_layout()

In [ ]:
#COMPARING CALCULATION METHODS
################################################################################

In [ ]:
# #METHOD ONE 
# t=100

# u_data=data['uinterp'].isel(time=t).data
# v_data=data['vinterp'].isel(time=t).data
# qv_data=data['qv'].isel(time=t).data
# rho_data=data['rho'].isel(time=t).data

# print('calculating convergence and taking mean')
# one=u_data*Ddx_3D(rho_data*qv_data,1000)
# two=v_data*Ddy_3D(rho_data*qv_data,1000)

# four=rho_data*qv_data*Ddx_3D(u_data,1000)
# five=rho_data*qv_data*Ddy_3D(v_data,1000)

# conv=-(one+two)-(four+five)
# conv[z]

In [ ]:
# #Method 2
# u_data=data['uinterp'].isel(time=t).data
# v_data=data['vinterp'].isel(time=t).data
# qv_data=data['qv'].isel(time=t).data
# rho_data=data['rho'].isel(time=t).data


# print('calculating convergence and taking mean')
# one=Ddx_3D(rho_data*qv_data*u_data,1000)
# two=Ddy_3D(rho_data*qv_data*v_data,1000)
# conv2=-(one+two)
# conv2[z]

In [ ]:
# arr=(conv-conv2)[z]
# plt.contourf(arr)
# plt.colorbar()